In [1]:
%pip install pandas numpy networkx matplotlib scikit-learn --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Preprocessing

In [1]:
path = "collected_posts.csv"

In [20]:
import pandas as pd
import networkx as nx
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Carica i dati dal CSV
df = pd.read_csv(path)

# Seleziona le colonne rilevanti
columns_to_keep = ['Username', 'User ID', 'Post ID', 'Like Count', 'Caption Text', 'Caption Created At (UTC)']
df_clean = df[columns_to_keep].copy()

# Converti 'Caption Created At (UTC)' in formato datetime
df_clean['Caption Created At (UTC)'] = pd.to_datetime(df_clean['Caption Created At (UTC)'], errors='coerce')

# Rimuovi righe con valori mancanti
df_clean = df_clean.dropna(subset=['User ID', 'Post ID', 'Like Count'])
df_clean = df_clean.drop_duplicates(subset='Post ID')

# Identificazione degli utenti influenti

In [22]:
# Creazione del grafo delle interazioni
G = nx.DiGraph()

# Aggiungi nodi e archi in base ai like
for _, row in df_clean.iterrows():
    user_id = row['User ID']
    post_id = f'post_{row["Post ID"]}'
    like_count = row['Like Count']
    
    G.add_node(user_id)
    G.add_edge(user_id, post_id, weight=like_count)

#### Misure di centralità

In [24]:
# Degree Centrality
degree_centrality = nx.degree_centrality(G)

# Closeness Centrality
closeness_centrality = nx.closeness_centrality(G)

# Betweenness Centrality
betweenness_centrality = nx.betweenness_centrality(G, normalized=True, weight='weight')

# PageRank
pagerank = nx.pagerank(G, alpha=0.85)

# Katz Centrality
katz_centrality = nx.katz_centrality(G, alpha=0.1, beta=1.0, max_iter=1000, tol=1e-06)

# Bonacich Centrality (classic power centrality)
# NetworkX non implementa direttamente la Bonacich Centrality. Tuttavia, puoi utilizzare la centralità di potenza (power centrality)
# che è una generalizzazione di quella di Bonacich.
# Per implementare Bonacich centrality, dobbiamo definire la formula o usare librerie esterne, ma qui applichiamo power centrality.
eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000)

# HITS Algorithm
hits_hub, hits_authority = nx.hits(G, max_iter=1000, tol=1e-08)

# Mostra i top 10 utenti per ciascuna metrica
print("Top 10 utenti per Degree Centrality:")
print(sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:10])

print("\nTop 10 utenti per Closeness Centrality:")
print(sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:10])

print("\nTop 10 utenti per Betweenness Centrality:")
print(sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:10])

print("\nTop 10 utenti per PageRank:")
print(sorted(pagerank.items(), key=lambda x: x[1], reverse=True)[:10])

print("\nTop 10 utenti per Eigenvector Centrality:")
print(sorted(eigenvector_centrality.items(), key=lambda x: x[1], reverse=True)[:10])

print("\nTop 10 utenti per Katz Centrality:")
print(sorted(katz_centrality.items(), key=lambda x: x[1], reverse=True)[:10])

#print("\nTop 10 utenti per Bonacich (Power) Centrality:")
#print(sorted(power_centrality.items(), key=lambda x: x[1], reverse=True)[:10])

print("\nTop 10 utenti per HITS Hub Scores:")
print(sorted(hits_hub.items(), key=lambda x: x[1], reverse=True)[:10])

print("\nTop 10 utenti per HITS Authority Scores:")
print(sorted(hits_authority.items(), key=lambda x: x[1], reverse=True)[:10])


Top 10 utenti per Degree Centrality:
[(62926993725, 0.011049723756906077), (58605518020, 0.007366482504604052), (59654151507, 0.007366482504604052), (68680407401, 0.006445672191528545), (12884127867, 0.0055248618784530384), (65293283875, 0.0055248618784530384), (22148830740, 0.004604051565377533), (2040587559, 0.003683241252302026), (2129098119, 0.003683241252302026), (45432108355, 0.0027624309392265192)]

Top 10 utenti per Closeness Centrality:
[('post_3474452509664264656', 0.0009208103130755065), ('post_3474350927834661356', 0.0009208103130755065), ('post_3473932439324360592', 0.0009208103130755065), ('post_3473768383085032947', 0.0009208103130755065), ('post_3474245558295728334', 0.0009208103130755065), ('post_3473630813577459268', 0.0009208103130755065), ('post_3473576320682720960', 0.0009208103130755065), ('post_3473565258301806527', 0.0009208103130755065), ('post_3446085903746401933', 0.0009208103130755065), ('post_3425126533460802710', 0.0009208103130755065)]

Top 10 utenti per 

# Clustering and Community Detection

In [8]:
#from networkx.algorithms.community import modularity_max
# Modularity-based community detection
#modular_communities = modularity_max(G)

from networkx.algorithms.community import girvan_newman

# Perform divisive clustering using betweenness-based method
gn_communities = list(girvan_newman(G))


# Diffusion Modeling: Independent Cascade (IC) and Linear Threshold (LT)

In [25]:
def independent_cascade(G, seeds, steps=5):
    active = set(seeds)
    new_active = set(seeds)
    for _ in range(steps):
        next_active = set()
        for node in new_active:
            for neighbor in G.neighbors(node):
                if neighbor not in active and np.random.rand() < G[node][neighbor].get('weight', 0.1):
                    next_active.add(neighbor)
        new_active = next_active - active
        active.update(new_active)
        if not new_active:
            break
    return active


In [26]:
def linear_threshold(G, seeds, steps=5):
    active = set(seeds)
    thresholds = {n: np.random.uniform(0, 1) for n in G.nodes()}
    for _ in range(steps):
        new_active = set()
        for node in G.nodes():
            if node not in active:
                neighbors = list(G.neighbors(node))
                influence = sum(1 for n in neighbors if n in active)
                if influence / len(neighbors) > thresholds[node]:
                    new_active.add(node)
        active.update(new_active)
        if not new_active:
            break
    return active


# Combining Models and Community Detection for Propagation Study

In [36]:
# Apply the Girvan-Newman algorithm for community detection
comp = nx.algorithms.community.girvan_newman(G)
communities = next(comp)  # Extract the first level of communities

seeds = []
for community in communities:
    subgraph = G.subgraph(community)
    betweenness = nx.betweenness_centrality(subgraph)
    leader = max(betweenness, key=betweenness.get)  # Node with the highest centrality in the community
    seeds.append(leader)

for community in communities:
    subgraph = G.subgraph(community)
    
    # Ensure that seeds are in the subgraph
    subgraph_seeds = [s for s in seeds if s in subgraph.nodes]
    
    # Run the Independent Cascade model on the subgraph
    if subgraph_seeds:  # Only run if there are valid seeds
        active_nodes = independent_cascade(subgraph, subgraph_seeds)

In [38]:
from sklearn.metrics import silhouette_score
import numpy as np

# Create labels for silhouette score
labels = {}
for i, community in enumerate(communities):
    for node in community:
        labels[node] = i

# Convert the labels dictionary to a list of labels corresponding to the graph nodes
labels_list = [labels.get(node) for node in G.nodes()]

# Ensure the graph is undirected (if you have a directed graph)
if nx.is_directed(G):
    G = G.to_undirected()

# Check if the graph is connected
if not nx.is_connected(G):
    # Extract the largest connected component
    largest_component = max(nx.connected_components(G), key=len)
    G = G.subgraph(largest_component).copy()

    # Filter labels for the largest component
    labels_list = [labels[node] for node in G.nodes()]

# Compute the shortest path distance matrix
shortest_path_lengths = dict(nx.all_pairs_shortest_path_length(G))

# Convert the shortest path lengths into a distance matrix
n = len(G.nodes())
dist_matrix = np.zeros((n, n))

node_list = list(G.nodes())
node_index = {node: idx for idx, node in enumerate(node_list)}

for i, node1 in enumerate(node_list):
    for j, node2 in enumerate(node_list):
        if node1 == node2:
            dist_matrix[i, j] = 0  # distance to self is 0
        else:
            dist_matrix[i, j] = shortest_path_lengths[node1].get(node2, np.inf)  # inf if no path exists

# Calculate silhouette score based on the distance matrix
silhouette_avg = silhouette_score(dist_matrix, labels_list, metric='precomputed')
print("Silhouette Score:", silhouette_avg)


Silhouette Score: 0.008771929824561353
